### Задача:
На сайте запущен А/В тест с целью увеличить доход. В приложенном excel файле вы найдете сырые данные по результатам эксперимента – user_id, тип выборки variant_name и доход принесенный пользователем revenue. \
Проанализируйте результаты эксперимента и напишите свои рекомендации менеджеру.

In [1]:
from typing import Union
from tqdm import tqdm

import pandas as pd
import numpy as np

from scipy import stats
from statsmodels.stats.meta_analysis import effectsize_smd
from statsmodels.stats import proportion
from statsmodels.stats.power import tt_ind_solve_power
from statsmodels.stats.power import zt_ind_solve_power

In [2]:
### Чтение файла с данными

df = pd.read_excel('gb_sem_8_hm.xlsx')
df.head()

C:\Users\Андрей\AppData\Roaming\Python\Python310\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


,USER_ID,VARIANT_NAME,REVENUE
0,737,variant,0.0
1,2423,control,0.0
2,9411,control,0.0
3,7311,control,0.0
4,6174,variant,0.0


#### Исследование "сырых" данных

In [3]:
### Описательная статистика

df.describe()

,USER_ID,REVENUE
count,10000.000000,10000.000000
mean,4981.080200,0.099447
std,2890.590115,2.318529
min,2.000000,0.000000
25%,2468.750000,0.000000
50%,4962.000000,0.000000
75%,7511.500000,0.000000
max,10000.000000,196.010000


In [4]:
### Просмотр данных о типах переменных

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   USER_ID       10000 non-null  int64  
 1   VARIANT_NAME  10000 non-null  object 
 2   REVENUE       10000 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 234.5+ KB


In [5]:
### Просмотр данных о пропущенных значениях

df.isna().sum()

USER_ID         0
VARIANT_NAME    0
REVENUE         0
dtype: int64

In [6]:
### Описательная статистика для контрольного и экспериментального наборов данных

df.groupby('VARIANT_NAME').describe()

USER_ID                                                          \
               count         mean          std  min     25%     50%      75%   
VARIANT_NAME                                                                   
control       4984.0  4989.436798  2905.145109  2.0  2466.0  4964.5  7576.25   
variant       5016.0  4972.776914  2876.320625  3.0  2476.5  4958.5  7415.25   

                      REVENUE                                                  
                  max   count      mean       std  min  25%  50%  75%     max  
VARIANT_NAME                                                                   
control       10000.0  4984.0  0.129013  3.007524  0.0  0.0  0.0  0.0  196.01  
variant       10000.0  5016.0  0.070070  1.314802  0.0  0.0  0.0  0.0   58.63

In [7]:
#### Создание таблицы с повторяющимися USER_ID в контрольной и экспериментальной выборках

Duplicated_Table = df. \
    groupby("USER_ID", as_index = False). \
    agg({"VARIANT_NAME" : pd.Series.nunique})
print(Duplicated_Table)

      USER_ID  VARIANT_NAME
0           2             1
1           3             2
2           4             1
3           5             1
4           6             1
...       ...           ...
6319     9993             1
6320     9995             1
6321     9996             2
6322     9998             1
6323    10000             2

[6324 rows x 2 columns]


## Удаление некорректных строк (наличие одного USER_ID в обеих группах эксперимента)

In [8]:
df_clean = df[-df["USER_ID"].isin(Duplicated_Table[Duplicated_Table["VARIANT_NAME"] > 1]["USER_ID"])]
df_clean

##### Теперь в наборе данных есть повторяющиеся значения USER_ID только либо в контрольной группе
##### либо в экспериментальной группе. 

,USER_ID,VARIANT_NAME,REVENUE
0,737,variant,0.0
4,6174,variant,0.0
5,2380,variant,0.0
7,9168,control,0.0
9,7548,control,0.0
...,...,...,...
9993,2400,variant,0.0
9994,3129,control,0.0
9996,502,variant,0.0
9998,7741,control,0.0


### Объединение повторяющихся USER_ID в рамках одного типа эксперимента.
Т.к. в задании не указано откуда в наборе данных есть повторяющиеся значения USER_ID в рамках одного типа эксперимента, то я предполагаю, что это, действительно, один и тот же пользователь (только в разные периоды времени, например, день, неделя, месяц) делал покупки и его ID затем попало в контретную группу АВ-теста. Поэтому, чтобы не потелять данные, я предлагаю объединить одинаковые USER_ID и суммировать их платежи.

In [9]:
#### Работа с контрольной группой

df_controls = df_clean[df_clean["VARIANT_NAME"] == "control"]
df_controls

,USER_ID,VARIANT_NAME,REVENUE
7,9168,control,0.0
9,7548,control,0.0
10,8395,control,0.0
11,1184,control,0.0
12,7165,control,0.0
...,...,...,...
9986,2161,control,0.0
9988,428,control,0.0
9994,3129,control,0.0
9998,7741,control,0.0


In [10]:
#### Просмотр записей в контрольной группе с ненулевыми платежами

df_controls[df_controls["REVENUE"] > 0]
df_controls[df_controls["REVENUE"] > 0]["REVENUE"].sum()

470.55999999999995

In [11]:
#### Объединение повторяющихся USER_ID в рамках контрольной группы

df_controls_clean = df_controls.groupby(['USER_ID'], as_index=False).agg({'REVENUE': 'sum'})
print(df_controls_clean.shape)
print(df_controls_clean.REVENUE.sum())

#### Количество строк уменьшилось, а сумма доходов не изменилась. Следовательно объединение
####  по USER_ID прошло правильно.


(2390, 2)
470.56


In [12]:
#### Работа с экспериментальной группой

df_variants = df_clean[df_clean["VARIANT_NAME"] == "variant"]
df_variants

,USER_ID,VARIANT_NAME,REVENUE
0,737,variant,0.0
4,6174,variant,0.0
5,2380,variant,0.0
14,3489,variant,0.0
16,7549,variant,0.0
...,...,...,...
9985,8509,variant,0.0
9987,8753,variant,0.0
9991,8864,variant,0.0
9993,2400,variant,0.0


In [13]:
#### Просмотр записей в экспериментальной группе с ненулевыми платежами

df_variants[df_variants["REVENUE"] > 0]
df_variants[df_variants["REVENUE"] > 0]["REVENUE"].sum()

179.32

In [14]:
#### Объединение повторяющихся USER_ID в рамках экспериментальной группы

df_variants_clean = df_variants.groupby(['USER_ID'], as_index=False).agg({'REVENUE': 'sum'})
print(df_variants_clean.shape)
print(df_variants_clean.REVENUE.sum())

#### Количество строк уменьшилось, а сумма доходов не изменилась. Следовательно объединение
####  по USER_ID прошло правильно.


(2393, 2)
179.32


In [15]:
#### Проверка отсутствия в группах повторяющихся значений USER_ID в группах и между группами

print(df_controls_clean[df_controls_clean.duplicated('USER_ID')])
print(df_variants_clean[df_variants_clean.duplicated('USER_ID')])

len(set(df_controls_clean.USER_ID).intersection(set(df_variants_clean.USER_ID)))

#### Повторяющихся USER_ID нет ни в группах, ни между групп в данных для AB-теста

Empty DataFrame
Columns: [USER_ID, REVENUE]
Index: []
Empty DataFrame
Columns: [USER_ID, REVENUE]
Index: []


0

### Анализ результатов АВ-теста по метрике REVENUE


In [21]:
#### Просмотр описательной статистики очищенных выборок

df_controls_clean.describe()

,USER_ID,REVENUE
count,2390.000000,2390.000000
mean,5020.881590,0.196887
std,2904.850992,4.172201
min,2.000000,0.000000
25%,2517.250000,0.000000
50%,5012.500000,0.000000
75%,7616.000000,0.000000
max,9998.000000,196.010000


In [22]:
df_variants_clean.describe()

,USER_ID,REVENUE
count,2393.000000,2393.000000
mean,4967.943168,0.074935
std,2892.745368,0.858207
min,4.000000,0.000000
25%,2435.000000,0.000000
50%,4955.000000,0.000000
75%,7379.000000,0.000000
max,9995.000000,23.040000


In [17]:
#### проверка тестом Шапиро-Уилка нормальности распределения в контрольной группе

W, p_value = stats.shapiro(df_controls_clean.REVENUE)
print("W = ", W)
print("p-value = ", p_value)

### Вывод: распределение ненормальное, т.к. p-value < 0.05

W =  0.021338164806365967
p-value =  0.0


In [18]:
#### проверка тестом Шапиро-Уилка нормальности распределения в экспериментальной группе

W, p_value = stats.shapiro(df_variants_clean.REVENUE)
print("W = ", W)
print("p-value = ", p_value)

### Вывод: распределение ненормальное, т.к. p-value < 0.05

W =  0.06145977973937988
p-value =  0.0


#### Соответственно, нужно применять либо непараметрические критерии, либо бутстреп с получением доверительного интервала для p-value разницы контрольной и экспериментальной групп.

#### Применение статистики Манна-Уитни

In [27]:
U, p_value = stats.mannwhitneyu(x = df_controls_clean.REVENUE, y = df_variants_clean.REVENUE)
print("U = ", U)
print("p-value = ", p_value)

### Вывод: группы статистически значимо не различаются, т.к. p-value > 0.05 

U =  2874161.0
p-value =  0.2104875008271777


#### Применение бутстрепа

In [19]:
def continious_result(control: pd.DataFrame,
                      treatment: pd.DataFrame,
                      column: str,
                      n_iters: int = 10_000) -> pd.DataFrame:
    # Статистика по выборкам
    size = control.loc[:, column].shape[0]
    
    control_mean = control.loc[:, column].mean()
    treatment_mean = treatment.loc[:, column].mean()
    
    control_std = control.loc[:, column].std(ddof=1)
    treatment_std = treatment.loc[:, column].std(ddof=1)
    
    # Бутсрап
    booted_diff = []
    for _ in tqdm(range(n_iters)):
        control_sample = control.loc[:, column].sample(n=size, replace=True).values
        treatment_sample = treatment.loc[:, column].sample(n=size, replace=True).values
        booted_diff.append(np.mean(control_sample - treatment_sample))
    
    # Считаем статистику после бустрапа
    md_ci, std_ci = np.mean(booted_diff), np.std(booted_diff, ddof=1)
    left_ci, right_ci = np.percentile(booted_diff, [2.5, 97.5])
    p_value_ci = 2 * (1 - stats.norm.cdf(np.abs(md_ci / std_ci)))
    
    # Считаем мощность эксперимента
    effect_size, _ = effectsize_smd(mean1=treatment_mean, sd1=treatment_std, nobs1=size,
                                    mean2=control_mean, sd2=control_std, nobs2=size)
    power = tt_ind_solve_power(effect_size=effect_size,
                               nobs1=size,
                               alpha=.05,
                               power=None,
                               ratio=1)
    # Формируем отчёт 
    result = pd.DataFrame({'effect_size': effect_size,
                           'alpha': p_value_ci, 
                           'beta': (1-power),
                           'CI': f'[{np.round(left_ci, 3)}, {np.round(right_ci, 3)}]',
                           'difference': md_ci,},
                          index=[column]) 
    return result

In [20]:
continious_result(control = df_controls_clean, treatment = df_variants_clean, column = 'REVENUE')

100%|██████████| 10000/10000 [00:04<00:00, 2125.23it/s]


,effect_size,alpha,beta,CI,difference
REVENUE,-0.040483,0.159944,0.712143,"[-0.002, 0.322]",0.121864


In [ ]:
### Вывод: группы статистически значимо не различаются, т.к. p-value (alpha) > 0.05
###  и доверительный интервал включает ноль.

### Окончательный вывод: 
#### Результаты непараметрических тестов и доверительных интервалов совпадают в части статистической незначимости различий между контрольной и экспериментальной группами. Таким образом, предложенные изменения не увеличат доход.